In [8]:
from pyspark.sql import SparkSession
from delta import *
from minio import Minio
import pyspark.sql.functions as F
import os
import pandas as pd

In [9]:
#CONFIGURAÇÕES SPARK
spark = (
    SparkSession.builder.master("local[*]").appName("csvToParquet")
    .config("spark.executor.memory", "16g")
    #HADOOP
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    #DELTA
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.jars", "jars/aws-java-sdk-bundle-1.11.1026.jar, \
            jars/hadoop-aws-3.2.0.jar")
    .getOrCreate()
)


In [10]:
#CONFIGURAÇÕES CLIENT DO MINIO
client=Minio("localhost:9000",
             access_key="minioadmin",
             secret_key="minioadmin",
             secure=False
            )


In [11]:
bucket_name = 'raw'
objects=client.list_objects(bucket_name, prefix='CAPES/', recursive=True)
csvs = []
for object in objects:
    csvs.append(object.object_name)

#LISTANDO OS DATAFRAMES PRESENTES NO BUCKET RAW DO MinIO
count = 0
for csv in csvs:
    print(f'{count} - {csv}')
    count+=1

0 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022 Bibliográfica Subtipo Livro.csv
1 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Artigo em Jornal ou Revista.csv
2 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Outro.csv
3 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Aplicativo.csv
4 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Produto.csv
5 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnic

In [12]:
#FUNÇÃO DE FILTRO PELA COLUNA ESPECIFICA USANDO A KEY ESPECIFICA
#OS PARAMETROS SAO: 
#NOME DO ARQUIVO NA DATABASE = name_database
#A DATABASE EM SI = database
#O CLIENT DO MINIO = client
#A COLUNA ESPECIFICA = column_key
#A KEY ESPECIFICA = key
def filterByColumn(name_database, database, client, column_key, key):
    #PARA CADA COLUNA DA DATABASE, EU PERCORRO ATE ENCONTRAR A QUE EU QUERO, SENAO NAO FAZ NADA
    for column_actual in database.columns:
        if column_actual == column_key:
            #SE EU ACHO ENTAO FILTRO USANDO OS PARAMETROS PASSADOS
            database = database.filter(F.col(column_key).like(key))
            #APOS USO O PANDAS PARA CONVERTE-LO EM CSV NOVAMENTE PARA CRIAR UM ARQUIVO TEMPORARIO PARA CONSEGUIR USAR O MINIO
            break
    database.toPandas().to_csv('temp.csv', sep=';', index=False)
    client.fput_object('bronze',f"{name_database}", 'temp.csv')
    #REMOVO O TEMPORARIO E FINALIZO A FUNÇÃO POIS JA FOI FEITO AS ALTERAÇÕES
    os.remove('temp.csv')
    
#A FUNÇÃO MODIFICA DIRETAMENTE O BANCO DE DADOS, SE QUER SALVAR AS ALTERAÇÕES EM OUTRO LUGAR ENTÃO APENAS MODIFIQUE A LINHA
#client.fput_object('raw',f"{name_database}", 'temp.csv'), COLOCANDO O LUGAR ESPECIFICO A SER SALVO, EXEMPLO SE QUER COLOCAR NUMA
#PASTA CHAMADA UFMA, ENTAO: client.fput_object('raw',f"UFMA/{name_database}", 'temp.csv')
#OS ARQUIVOS SEMPRE SERAO SALVOS EM UMA SUBPASTA CHAMADA CAPES/ POIS OS MESMOS JA POSSUEM ELA NO NOME

In [13]:
#NOS FILTROS, FORAM USADOS QUATRO COLUNAS POIS VARIOS ARQUIVOS POSSUIAM NOMENCLATURAS DIFERENTES PARA A AREA DE INTERESSE
for csv_number in range(0, len(csvs)):
    database = spark.read.format('csv').option('header', 'true').option('sep', ';')\
        .load(f's3a://raw/{csvs[csv_number]}')
    filterByColumn(csvs[csv_number], database, client, "SG_ENTIDADE_ENSINO", "%UFMA%")
    print(f"{csvs[csv_number]} OK")
bucket_name = 'bronze'

CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022 Bibliográfica Subtipo Livro.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Artigo em Jornal ou Revista.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Outro.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Aplicativo.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Produto.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo

CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021, Bibliográfica Subtipo Artigo em Periódico.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021, Bibliográfica Subtipo Trabalho em Anais.csv OK


CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2022, Bibliográfica Subtipo Artigo em Periódico.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2022, Bibliográfica Subtipo Trabalho em Anais.csv OK


CAPES/2021-a-2024-catalogo-de-teses-e-dissertacoes-brasil/Ano 2021.csv OK


CAPES/2021-a-2024-catalogo-de-teses-e-dissertacoes-brasil/Ano 2022.csv OK
CAPES/2021-a-2024-cursos-da-pos-graduacao-stricto-sensu-no-brasil/Ano 2021.csv OK
CAPES/2021-a-2024-cursos-da-pos-graduacao-stricto-sensu-no-brasil/Ano 2022.csv OK


...

CAPES/2021-a-2024-membros-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano 2021.csv OK


CAPES/2021-a-2024-membros-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano 2022.csv OK
CAPES/2021-a-2024-producao-intelectual-de-pos-graduacao-stricto-sensu-no-brasil/Anos 2021 e 2022 - Tipo Bibliográfica Subtipo Livro.csv OK
CAPES/2021-a-2024-producao-intelectual-de-pos-graduacao-stricto-sensu-no-brasil/Anos 2021 e 2022 - Tipo Bibliográfica Subtipo Outro.csv OK
CAPES/2021-a-2024-producao-intelectual-de-pos-graduacao-stricto-sensu-no-brasil/Anos 2021 e 2022 - Tipo Bibliográfica Subtipo Trabalho em Anais.csv OK
CAPES/2021-a-2024-producao-intelectual-de-pos-graduacao-stricto-sensu-no-brasil/Anos 2021 e 2022 - Tipo Técnica Subtipo Artigo em Jornal ou Revista.csv OK
CAPES/2021-a-2024-producao-intelectual-de-pos-graduacao-stricto-sensu-no-brasil/Anos 2021 e 2022 - Tipo Técnica Subtipo Desenvolvimento de Aplicativo.csv OK
CAPES/2021-a-2024-producao-intelectual-de-pos-graduacao-stricto-sensu-no-brasil/Anos 2021 e 2022 - Tipo Técnica Subtipo Desenvolvimento de Produto.csv

CAPES/2021-a-2024-producao-intelectual-de-pos-graduacao-stricto-sensu-no-brasil/Anos 2021 e 2022 – Tipo Bibliográfica Subtipo Artigo em Periódico.csv OK
CAPES/2021-a-2024-programas-da-pos-graduacao-stricto-sensu-no-brasil/Ano 2021.csv OK
CAPES/2021-a-2024-programas-da-pos-graduacao-stricto-sensu-no-brasil/Ano 2022.csv OK


CAPES/2021-a-2024-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano 2021.csv OK


CAPES/2021-a-2024-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano 2022.csv OK


In [14]:
for csv_number in range(0, len(csvs)):
    database = spark.read.format('csv').option('header', 'true').option('sep', ';')\
        .load(f's3a://{bucket_name}/{csvs[csv_number]}')
    filterByColumn(csvs[csv_number], database, client, "NM_AREA_CONHECIMENTO", "%COMPUTA%")
    print(f"{csvs[csv_number]} OK")

CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022 Bibliográfica Subtipo Livro.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Artigo em Jornal ou Revista.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Outro.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Aplicativo.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Produto.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo

In [15]:
for csv_number in range(0, len(csvs)):
    database = spark.read.format('csv').option('header', 'true').option('sep', ';')\
        .load(f's3a://{bucket_name}/{csvs[csv_number]}')
    filterByColumn(csvs[csv_number], database, client, "NM_AREA_CONCENTRACAO", "%COMPUTA%")
    print(f"{csvs[csv_number]} OK")

CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022 Bibliográfica Subtipo Livro.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Artigo em Jornal ou Revista.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Outro.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Aplicativo.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Produto.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo

In [16]:
for csv_number in range(0, len(csvs)):
    database = spark.read.format('csv').option('header', 'true').option('sep', ';')\
        .load(f's3a://{bucket_name}/{csvs[csv_number]}')
    filterByColumn(csvs[csv_number], database, client, "NM_AREA_AVALIACAO", "%COMPUTA%")
    print(f"{csvs[csv_number]} OK")

CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022 Bibliográfica Subtipo Livro.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Artigo em Jornal ou Revista.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Outro.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Aplicativo.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Produto.csv OK
CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo

In [17]:
database = spark.read.format('csv').option('header', 'true').option('sep', ';')\
        .load(f's3a://{bucket_name}/{csvs[0]}')

In [18]:
for csv_number in range(0, len(csvs)):
    database = spark.read.format('csv').option('header', 'true').option('sep', ';')\
        .load(f's3a://{bucket_name}/{csvs[csv_number]}')
    print(str(csv_number) + " " + csvs[csv_number],database.count(), '\n')

0 CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022 Bibliográfica Subtipo Livro.csv 13 

1 CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Artigo em Jornal ou Revista.csv 0 

2 CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Bibliográfica Subtipo Outro.csv 0 

3 CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Aplicativo.csv 21 

4 CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 2022, Técnica Subtipo Desenvolvimento de Produto.csv 0 

5 CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil/Ano Base - 2021 a 20

In [45]:
n = 30
print(csvs[n])
database = spark.read.format('csv').option('header', 'true').option('sep', ';')\
        .load(f's3a://{bucket_name}/{csvs[n]}')
print(*database.columns, sep='\n')

CAPES/2021-a-2024-docentes-da-pos-graduacao-stricto-sensu-no-brasil/Ano 2021.csv
AN_BASE
CD_AREA_AVALIACAO
NM_AREA_AVALIACAO
NM_GRANDE_AREA_CONHECIMENTO
NM_AREA_CONHECIMENTO
CD_PROGRAMA_IES
NM_PROGRAMA_IES
NM_GRAU_PROGRAMA
NM_MODALIDADE_PROGRAMA
CD_CONCEITO_PROGRAMA
CD_ENTIDADE_CAPES
CD_ENTIDADE_EMEC
SG_ENTIDADE_ENSINO
NM_ENTIDADE_ENSINO
DS_DEPENDENCIA_ADMINISTRATIVA
CS_STATUS_JURIDICO
NM_MUNICIPIO_PROGRAMA_IES
SG_UF_PROGRAMA
NM_REGIAO
ID_PESSOA
TP_DOCUMENTO_DOCENTE
NR_DOCUMENTO_DOCENTE
NM_DOCENTE
AN_NASCIMENTO_DOCENTE
DS_FAIXA_ETARIA
DS_TIPO_NACIONALIDADE_DOCENTE
NM_PAIS_NACIONALIDADE_DOCENTE
DS_CATEGORIA_DOCENTE
DS_TIPO_VINCULO_DOCENTE_IES
DS_REGIME_TRABALHO
CD_CAT_BOLSA_PRODUTIVIDADE
IN_DOUTOR
AN_TITULACAO
NM_GRAU_TITULACAO
CD_AREA_BASICA_TITULACAO
NM_AREA_BASICA_TITULACAO
SG_IES_TITULACAO
NM_IES_TITULACAO
NM_PAIS_IES_TITULACAO
ID_ADD_FOTO_PROGRAMA
ID_ADD_FOTO_PROGRAMA_IES


In [21]:
arquivos_por_pasta = {}

for caminho in csvs:
    _, pasta, arquivo = caminho.split('/')
    if pasta not in arquivos_por_pasta:
        arquivos_por_pasta[pasta] = []
    arquivos_por_pasta[pasta].append(arquivo)
